In [27]:
#___________________________________________________________________________________________________________________
get_ipython().magic('matplotlib notebook')
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import pyvista as pv
import vtk
import tripyview as tpv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# Parameters
mesh_path ='/home/pscholz/FESOM2.0/meshes/core2/'
save_path = None #'~/figures/test_papermill/'
save_fname= None

#_____________________________________________________________________________________
which_cycl= 3
which_mode= '3dsphere'

#_____________________________________________________________________________________
input_paths= list()
input_paths.append(None)

input_names= list()
input_names.append('resolution')

vname     = 'depth'
year      = [1979,2019]
mon, day, record, box, depth = None, None, None, None, 100

#_____________________________________________________________________________________
# do anomaly plots in case ref_path is not None
ref_path  = None #'/home/ollie/pscholz/results/trr181_tke_ctrl_ck0.1/' # None
ref_name  = None #'TKE, ck=0.1' # None
ref_year  = None # [2009,2019]
ref_mon, ref_day, ref_record = None, None, None

#_____________________________________________________________________________________
do_clim   = False
which_clim= 'phc3'
clim_path = '/work/ollie/pscholz/INIT_HYDRO/phc3.0/phc3.0_annual.nc'

#_____________________________________________________________________________________
cmap      = 'rygbw'
cnum      = 20
cref      = None
crange, cmin, cmax, cfac = None, None, None, None

#_____________________________________________________________________________________
ncolumn       = 1 
do_rescale    = True
which_dpi     = 300
#_____________________________________________________________________________________
do_potatoefac = 0.6
do_topo       = True


In [29]:
#___LOAD FESOM2 MESH___________________________________________________________________________________
mesh=tpv.load_mesh_fesom2(mesh_path, do_rot='None', focus=0, do_info=True, do_pickle=True,
                      do_earea=True, do_narea=True, do_eresol=[True,'mean'], do_nresol=[True,'eresol'])

#______________________________________________________________________________________________________
if which_cycl is not None: 
    for ii,ipath in enumerate(input_paths):
        if ipath is not None: 
            input_paths[ii] = os.path.join(ipath,'{:d}/'.format(which_cycl))
            print(ii, input_paths[ii])
    
    if ref_path is not None:
        ref_path = os.path.join(ref_path,'{:d}/'.format(which_cycl))
        print('R', ref_path)
#______________________________________________________________________________________________________
cinfo=dict({'cstr':cmap, 'cnum':cnum})     
if crange   is not None: cinfo['crange']=crange
if cmin     is not None: cinfo['cmin'  ]=cmin
if cmax     is not None: cinfo['cmax'  ]=cmax
if cref     is not None: cinfo['cref'  ]=cref
if cfac     is not None: cinfo['cfac'  ]=cfac
if ref_path is not None: cinfo['cref'  ]=0.0    
    
#______________________________________________________________________________________________________
# in case of diff plots
if ref_path is not None:
    if ref_year   is None: ref_year   = year
    if ref_mon    is None: ref_mon    = mon
    if ref_record is None: ref_record = record

 > found *.pckl file: /home/pscholz/FESOM2.0/meshes/core2
 > load  *.pckl file: tripyview_fesom2_core2.pckl
___FESOM2 MESH INFO________________________
 > path            = /home/pscholz/FESOM2.0/meshes/core2
 > id              = core2
 > do rot          = None
 > [al,be,ga]      = 50, 15, -90
 > do augmpbnd     = True
 > do cavity       = False
 > do lsmask       = True
 > do earea,eresol = True, True
 > do narea,nresol = True, True
___________________________________________
 > #node           = 126858
 > #elem           = 244659
 > #lvls           = 48
___________________________________________


In [30]:
#___LOAD FESOM2 REFERENCE DATA________________________________________________________________________
if ref_path is not None:
    if depth =='bottom':
        data_ref = tpv.load_data_fesom2(mesh, ref_path, vname=vname, year=ref_year, mon=ref_mon, day=ref_day, record=ref_record, 
                                      depth=None, descript=ref_name, do_info=False) 
        data_ref = data_ref.isel(nz=xr.DataArray(mesh.n_iz, dims='nod2'), nod2=xr.DataArray(range(0,mesh.n2dn), dims='nod2'))
    else:
        data_ref = tpv.load_data_fesom2(mesh, ref_path, vname=vname, year=ref_year, mon=ref_mon, day=ref_day, record=ref_record, 
                                      depth=depth, descript=ref_name, do_info=False) 

#___LOAD FESOM2 DATA___________________________________________________________________________________
data_list = list()
for datapath,descript in zip(input_paths, input_names): 
    print(datapath)
    if depth =='bottom':
        data = tpv.load_data_fesom2(mesh, datapath, vname=vname, year=year, mon=mon, day=day, record=record, 
                                      depth=None, descript=descript, do_info=False) 
        data = data.isel(nz=xr.DataArray(mesh.n_iz, dims='nod2'), nod2=xr.DataArray(range(0,mesh.n2dn), dims='nod2'))
    else:
        data = tpv.load_data_fesom2(mesh, datapath, vname=vname, year=year, mon=mon, day=day, record=record, 
                                      depth=depth, descript=descript, do_info=False) 
    #__________________________________________________________________________________________________    
    if ref_path is not None:
        data_list.append(do_anomaly(data, data_ref))  
    else:
        data_list.append(data)  
    del(data)
    
if ref_path is not None: del(data_ref)    
    
#___APPEND ABS CLIMATOLOGY_____________________________________________________________________________    
if (vname in ['temp', 'salt', 'pdens'] or 'sigma' in vname) and (depth is not 'bottom') and do_clim and (ref_path is None): 
    clim_vname = vname
    if   vname=='temp' and  which_clim.lower()=='woa18': clim_vname = 't00an1'
    elif vname=='salt' and  which_clim.lower()=='woa18': clim_vname = 's00an1'
    clim = tpv.load_climatology(mesh, clim_path, clim_vname, depth=depth)
    data_list.append(clim)    
    del(clim)

None


In [31]:
#
#
#___CREATE PYVISTA OCEAN MESH___________________________________________________________
meshpv_ocean = tpv.create_3d_ocean_mesh(mesh, data_list[0], potatoefac=do_potatoefac, variable=vname)

#
#
#___CREATE PYVISTA LAND MESH TO FILL HOLES______________________________________________
# # meshpv_land = create_3d_land_mesh(mesh)
meshpv_land = tpv.create_3d_land_mesh(mesh,
                                  potatoefac  = 0.25,
                                  resol       = 0.3,
                                  do_topo     = do_topo, 
                                  topo_path   = '/home/pscholz/AWI/AWI_DATA/data_ETOPO5/topo_1deg.nc',
                                  topo_varname= 'topo',
                                  topo_dimname= ['lon','lat'])
#
#
#___CREATE PYVISTA 3d COASTLINE, LONLAT GRID____________________________________________
points_coast         = tpv.create_3d_coastline(mesh)    
points_lonlat_grid   = tpv.create_3d_lonlat_grid(dlon=30,dlat=15,potatoefac=0.3,do_topo=do_topo)
points_0lon0lat_grid = tpv.create_3d_0lon0lat_grid(dlon=30,dlat=15,potatoefac=0.3,do_topo=do_topo)

 --> compute 3d ocean mesh
 --> compute 3d land mesh
(249274,)
 --> compute 3d coastline
 --> compute 3d longitude grid
 --> compute 3d latitude grid
 --> compute 3d equator & 0merid line


/home/pscholz/Python/tripyview/tripyview/sub_3dsphere.py:184: RuntimeWarning: invalid value encountered in arcsin
  meshpv_land.active_t_coords[:,1] = 0.5 + np.arcsin(zs)/np.pi


In [57]:
#___CREATE PYVISTA OCEAN MESH___________________________________________________________
do_mode    = 'interactive' # 'interactive', 'movie', 'print'
# do_mode    = 'movie' # 'interactive', 'movie', 'print'
# do_mode    = 'print' # 'interactive', 'movie', 'print'
# do_wsize   = [5000,5000] #determines resolution when printing
do_wsize   = [4000,4000] #determines resolution when printing
do_contour = False
do_gridline= True
do_smooth  = False
init_lon, init_lat, init_zoom = -30, 50, 1.75 #2

# lightning options
diffuse        = 1.0
specular       = 0.1 
specular_power = 2
ambient        = 0.1
metallic       = 1
roughness      = 0

# land texture options
do_landtexture= True
landcolor     = [0.6, 0.6, 0.6]
texture_file  = 'world.topo.bathy.200408.3x5400x2700.png'
texture_path  = os.path.join(tpv.__path__[0], 'backgrounds/')

#_______________________________________________________________________________________
# create custom colormap
vname = list(data_list[0].keys())[0]
cinfo = dict()
if any(x in vname for x in ['depth','topo','topography','zcoord']):  
    cinfo['crange'], cinfo['cstr'], cinfo['cnum'] = [0, 6000, 3000], 'wbgyr', 20
    
elif any(x in vname for x in ['nresol','n_resol','resolution','resol']):
    cinfo['cstr'  ] = 'rygbw'
    
cinfo = tpv.do_setupcinfo(cinfo, data_list[0], None, mesh, False)
clim = [cinfo['clevel'][0],cinfo['clevel'][-1]]

# create colorbar string
cbar_label, cbar_unit = None, None
if cbar_label is None     : cbar_label = data_list[0][vname].attrs['long_name']
if cbar_unit  is None     : cbar_label = cbar_label+' ['+data_list[0][ vname ].attrs['units']+']'
else                      : cbar_label = cbar_label+' ['+cbar_unit+']'
if 'str_ltim' in data_list[0][vname].attrs.keys(): cbar_label = cbar_label+'\n'+data_list[0][vname].attrs['str_ltim']
if 'str_ldep' in data_list[0][vname].attrs.keys(): cbar_label = cbar_label+data_list[0][vname].attrs['str_ldep']

#_______________________________________________________________________________________
# create pyvista plot object
# pv.set_plot_theme("default")
pv.set_plot_theme("document")
# pv.set_plot_theme("dark")
# pv.set_plot_theme("ParaView")

if do_mode=='print':
    pv.set_plot_theme("document")
    plt = pv.Plotter(notebook=True, off_screen=False, window_size=[1000,1000], 
                     multi_samples=4, line_smoothing=True, polygon_smoothing=True)
    plt.background_color=None
elif do_mode=='movie':
    do_wsize = [2000,2000]
    plt = pv.Plotter(notebook=False,window_size=do_wsize)    
elif do_mode=='interactive':
    do_wsize = [2000,2000]
    plt = pv.Plotter(notebook=True, window_size=do_wsize,
                    multi_samples=4, line_smoothing=True, polygon_smoothing=True)

#_______________________________________________________________________________________
# colorbar arguments
fsize_label, fsize_title = 12, 24
linew1 = 2
if   do_mode=='print'      : 
    fsize_label, fsize_title = fsize_label*6, fsize_title*4
    linew1 = 4
elif do_mode=='interactive': 
    fsize_label, fsize_title = fsize_label*2, np.int16(fsize_title*1.5)
    linew1 = 1
dict_sargs = dict(
             title          = cbar_label,    
             vertical       = True, height=0.75, position_x=0.875, position_y=0.125,
             shadow         = True,
             title_font_size= fsize_title,
             label_font_size= fsize_label,
             fmt            = "%4.0f",
             n_labels       = np.int8(len(cinfo['clevel'])),#n_labels=np.int8(len(clevel)/2)+1,
             n_colors       = cinfo['cmap'].N,
             outline        = False)

dict_annot=dict()

# _______________________________________________________________________________________
# plot 3d ocean mesh
print('--> draw ocean mesh')
plt.add_mesh(meshpv_ocean, 
             scalars=vname,
             clim=clim, cmap=cinfo['cmap'], n_colors=cinfo['cmap'].N,
             show_edges=False, 
             edge_color='black',
             show_scalar_bar=True, scalar_bar_args=dict_sargs,
             diffuse=diffuse, specular=specular, specular_power=specular_power, ambient=ambient, metallic=metallic, roughness=roughness, 
             lighting=True,
             annotations=dict_annot,
             smooth_shading=do_smooth)

if do_contour:
    contours = meshpv_ocean.contour(scalars='topo', isosurfaces=np.array([250,500,1000,1500,2000,3000,4000,5000,6000]))
    plt.add_mesh(contours, color="black", line_width=4, smooth_shading=True)
    del contours
    
#_______________________________________________________________________________________
# plot 3d land mesh
print('--> draw land mesh')
texture=None
if do_landtexture: 
    texture   = pv.read_texture(os.path.join(texture_path, texture_file))
    landcolor = [0.99, 0.99, 0.99]
    diffuse, specular, specular_power, ambient = 0.65, 0.2, 5, 0.65

plt.add_mesh(meshpv_land, color=landcolor, texture=texture, lighting=True, smooth_shading=do_smooth,
             diffuse=diffuse, specular=specular, specular_power=specular_power, ambient=ambient)
# if do_contour and do_topo: # --> needs some tweeking/smoothing of the contourslines 
#     contours = meshpv_land.contour(scalars='topo', isosurfaces=np.array([1000,2000,4000]))
#     plt.add_mesh(contours, color="w", line_width=3, smooth_shading=True)
#     del contours

plt.add_text('Patrick.Scholz@awi.de, FESOM2 developer team',position=[0.125*do_wsize[0], 0.125*do_wsize[1]], #position='upper_right', 
             font_size=16, color=[0.5, 0.5, 0.5], shadow=True)         

#_______________________________________________________________________________________
# plot 3d coast, lon-grid and lat-grid lines
print('--> draw coast lines')
for points in points_coast        : plt.add_lines(points, color='black',width=linew1*2)
if do_gridline: 
    print('--> draw grid lines ')    
    for points in points_lonlat_grid  : plt.add_lines(points, color='black', width=linew1)
    for points in points_0lon0lat_grid: plt.add_lines(points, color='black', width=linew1*2)

#_______________________________________________________________________________________
# do slider widget infrastructure for interactive plotting
R_earth    =  6371.0e3
if do_mode == 'interactive':
    wgap    = 0.05 
    wslider = (1-4*wgap)/3
    engine  = tpv.widget_lon_lat_zoom(plt, init_lon,init_lat,init_zoom)
    plt.add_slider_widget(lambda value: engine(plt, 'center_lon', int(value)), 
                          [-180, 180], value = int(init_lon), title='lon', style='modern',      
                          pointa=(1*wgap+0*wslider, .06), pointb=(1*wgap+1*wslider, .06))
    plt.add_slider_widget(lambda value: engine(plt, 'center_lat', int(value)),
                          [ -90,  90], value =  int(init_lat), title='lat', style='modern' ,  
                          pointa=(2*wgap+wslider, .06),   pointb=(2*wgap+2*wslider, .06))
    plt.add_slider_widget(lambda value: engine(plt, 'zoom_fac',   value),
                          [0.1,  10],   value =  init_zoom,  title='zoom_fac', style='modern', 
                          pointa=(3*wgap+2*wslider, .06), pointb=(3*wgap+3*wslider, .06))
    plt.show()
#     plt.show(jupyter_backend='static')
    # jupyter_backend='panel' is much faster for interactive mode (might require
    # conda install -c conda-forge pyvista panel) but does jet not allow for slider widgets
    # plt.show(jupyter_backend='panel', window_size=[900,900] )

#_______________________________________________________________________________________
# do animated mp4 of rotating earth 
elif do_mode =='movie':
    
    frame_nr    = 720
    frame_rate  = 16
    frame_wsize = [1008,1008]
    fname       = 'movie_rotate_3dglobe_'+do_var+'.mp4'
    plt.open_movie(fname, framerate=frame_rate)
    
    #___________________________________________________________________________________
    frame_lon = np.linspace(-30, 720-30, frame_nr+1)
    # frame_lat = -np.tanh(np.linspace(-360,360,360)*0.2)*45
    frame_lat = -np.cos(np.linspace(-180, 180, frame_nr+1)*np.pi/180)*45
    
    #___________________________________________________________________________________
    x,y,z = tpv.grid_cart3d(frame_lon[0], frame_lat[0], R_earth*10/2)
    plt.camera_position = [ np.array([x,y,z]), (0,0,0), (0,0,1)] 
    plt.show(auto_close=False, interactive_update=True, window_size=frame_wsize)
    for iframe in range(0,frame_nr):
            x,y,z = tpv.grid_cart3d(frame_lon[iframe], frame_lat[iframe], R_earth*10/2, is_deg=True)
            plt.camera_position = [ np.array([x,y,z]), (0,0,0), (0,0,1)] 
            plt.update()
#             plt.show(auto_close=False, interactive_update=True, window_size=frame_wsize)
            plt.write_frame()
    plt.close()
    
#_______________________________________________________________________________________
# do save as png
# https://nbviewer.org/github/banesullivan/damavand-volcano/blob/master/Damavand_Volcano.ipynb
elif do_mode =='print':   
    fname = 'image_3dsphere_'+vname+'.png'
    x,y,z = tpv.grid_cart3d(init_lon, init_lat, R_earth*10/1.8, is_deg=True)
    plt.camera_position = [ np.array([x,y,z]), (0,0,0), (0,0,1)] 
    print('--> print screenshot')
    plt.screenshot(filename=fname, window_size=do_wsize, transparent_background=True)
    plt.show(jupyter_backend='static')
#     plt.show(jupyter_backend='ipygany)
#     plt.show(jupyter_backend='pythreejs')
#     plt.show(jupyter_backend='ipyvtklink')
#     plt.show(jupyter_backend='panel')
#     plt.show(jupyter_backend='none')
#     plt.show()

--> draw ocean mesh
--> draw land mesh
--> draw coast lines
--> draw grid lines 


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)